# Real-Time Stream Viewer (HTTP)
the following function responds to HTTP requests with the list of last 10 processed twitter messages + sentiments in reverse order (newest on top), it reads records from the enriched stream, take the recent 10 messages, and reverse sort them. the function is using nuclio context to store the last results and stream pointers for max efficiency.<br> 

The code is automatically converted into a nuclio (serverless) function and and respond to HTTP requests<br>

the example demonstrate the use of `%nuclio` magic commands to specify environment variables, package dependencies,<br>configurations, and to deploy functions automatically onto a cluster.


## Initialize nuclio emulation, environment variables and configuration
use `# nuclio: ignore` for sections that don't need to be copied to the function

In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

## Nuclio function implementation
this function can run in Jupyter or in nuclio (real-time serverless)

In [60]:
import v3io.dataplane
import json
import os

def init_context(context):
    access_key = os.getenv('V3IO_ACCESS_KEY', None)
    setattr(context, 'container', os.getenv('V3IO_CONTAINER', 'users'))
    setattr(context, 'stream_path', os.getenv('STOCKS_STREAM',os.getenv('V3IO_USERNAME') + '/stocks/stocks_stream'))
    
    v3io_client = v3io.dataplane.Client(endpoint=os.getenv('V3IO_API', None), access_key=access_key)
    setattr(context, 'data', [])
    setattr(context, 'v3io_client', v3io_client) 
    setattr(context, 'limit', os.getenv('LIMIT', 10))
    
    
def handler(context, event):
    resp = context.v3io_client.seek_shard(container=context.container, path=f'{context.stream_path}/0', seek_type='EARLIEST')
    setattr(context, 'next_location', resp.output.location)
    resp = context.v3io_client.get_records(container=context.container, path=f'{context.stream_path}/0', location=context.next_location, limit=context.limit)
    context.logger.info('location: %s', context.next_location)

    for rec in resp.output.records:
        rec_data = rec.data.decode('utf-8')
        rec_json = json.loads(rec_data)
        context.data.append({'Time': rec_json['time'],
                             'Symbol': rec_json['symbol'],
                             'Sentiment': rec_json['sentiment'],
                             'Link': rec_json['link'],
                             'Content': rec_json['content']})

    context.data = context.data[-context.limit:]
    
    columns = [{'text': key, 'type': 'object'} for key in ['Time', 'Symbol', 'Sentiment', 'Link', 'Content']]
    data = [list(item.values()) for item in context.data]
    response = [{'columns': columns,
                'rows': data,
                'type': 'table'}]
    return response              

In [61]:
# nuclio: end-code

## Function invocation
the following section simulates nuclio function invocation and will emit the function results

In [62]:
# create a test event and invoke the function locally
init_context(context)
event = nuclio.Event(body='')
resp = handler(context, event)

Python> 2021-03-25 14:01:20,229 [info] location: AQAAAGYAAABHAEBeFwAAAA==


## Deploy a function onto a cluster
the `%nuclio deploy` command deploy functions into a cluster, make sure the notebook is saved prior to running it !<br>check the help (`%nuclio help deploy`) for more information

In [1]:
import mlrun
import os

# Export the bare function
fn = mlrun.code_to_function('stream-viewer',
                      handler='handler',kind="nuclio", image='mlrun/mlrun:0.6.5')

# Set parameters for current deployment
fn.set_envs({'V3IO_CONTAINER': 'users',
             'STOCKS_STREAM':  os.getenv('V3IO_USERNAME') + '/stocks/stocks_stream'})
fn.spec.max_replicas = 2
fn.apply(mlrun.platforms.v3io_cred())
fn.apply(mlrun.mount_v3io())
fn.spec.build.commands = ['pip install v3io']

fn.export('03-stream-viewer.yaml')

> 2021-09-29 06:03:07,038 [info] function spec saved to path: 03-stream-viewer.yaml


In [2]:
addr = fn.deploy(project="stocks-" + os.getenv('V3IO_USERNAME'))

> 2021-09-29 06:03:07,054 [info] Starting remote function deploy
2021-09-29 06:03:07  (info) Deploying function
2021-09-29 06:03:07  (info) Building
2021-09-29 06:03:07  (info) Staging files and preparing base images
2021-09-29 06:03:07  (info) Building processor image
2021-09-29 06:03:18  (info) Build complete
2021-09-29 06:03:26  (info) Function deploy complete
> 2021-09-29 06:03:26,280 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-stocks-dani-stream-viewer.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['default-tenant.app.dev8.lab.iguazeng.com:32523']}


In [ ]:
!curl {addr}